In [1]:
#!/usr/bin/evn python
#-*- coding: utf-8 -*-

import urllib
import urllib2
import json
import pandas as pd
import os
import pandas as pd
import numpy as np
#显示所有列
pd.set_option('display.max_columns', 10)
#显示所有行
pd.set_option('display.max_rows', 10)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',10000)

# load data

In [ ]:
# load data
data_set=pd.read_table('id_no_0106.txt',sep='\t',header=0,names=['mobile_hash','phone','id_initial','card_initial'])
data_set.head()

# 测试

In [ ]:
http://pay.autohome.com.cn/washing-number/realName/genHash.htm?_appid=qczf&consumerId=qcjr.dev.payment
    &custName=P116318760
    &idNo=P81495026
    &mobile=P16466104
    &cardNo=P294137118

In [9]:
# 身份证号测试
s='P81495026'

# 测试一个手机号试一下
url = '''http://pay.autohome.com.cn/washing-number/realName/genHash.htm?_appid=qczf&consumerId=qcjr.dev.payment&idNo=%(1)s'''%{'1':s}
req = urllib2.Request(url)
res_data = urllib2.urlopen(req)
json_str = res_data.read()
json_obj = json.loads(json_str)

print json_obj['idNo']


gHIUe3iE35cksqkMaXUz2tC9xp5ihRRuMD9/2qNh05wAdgAAAAE=


# 身份证号解密

In [37]:
# 筛选非空身份证号, 并整体去重
data_id=data_set[data_set.id_initial.notnull()][['mobile_hash','id_initial']].drop_duplicates()
data_id.head()

,mobile_hash,id_initial
0,0000f50fb74f427ffa60daf04e43effe67068da9d30c06ea911bdea7bbfa1bb6,P282227488
1,000223184d00256e5de3a0115d525895c47f4a9804e3cee3484fbf6dffd6f6b0,P304748806
2,00022aecf18524455efc768692309abc5cb5f613bf4e627d49ad27e293eef80e,P195738239
4,000641c2634b84ec40e7f56ae68128ddd30801acc0b4d7dceaf7c21bf2762406,P342373855
7,000b6d48b1f3bac3a4c817927da614d6aae73824e5ef9bbb296f9a4fb3124ff8,P193756379


In [38]:
data_id.shape

(21498, 2)

## 原始转可逆密文

In [ ]:
id_miwen=[]
for id_i in data_id['id_initial']:
    url = '''http://pay.autohome.com.cn/washing-number/realName/genHash.htm?_appid=qczf&consumerId=qcjr.dev.payment&idNo=%(1)s'''%{'1':id_i}
    req = urllib2.Request(url)
    res_data = urllib2.urlopen(req)
    json_str = res_data.read()
    json_obj = json.loads(json_str)
    id_miwen.append(json_obj['idNo'])


data_id['id_miwen']=id_miwen
data_id.head()

In [ ]:
data_id.head()

## 可逆密文urlencode

In [ ]:
data_id=data_id.reset_index()
id_urlencode=[]
for i in range(data_id.shape[0]):
    i_dict={}
    i_dict['name']=data_id['id_miwen'][i]
    id_urlencode.append(urllib.urlencode(i_dict))
    

data_id['id_urlencode']=id_urlencode

In [ ]:
# 去掉字典的key值
a=[]
for i in data_id['id_urlencode']:
    a.append(i.split('=')[1])


data_id['id_urlencode_last']=a

## 可逆密文转明文

In [ ]:
# http://se-j.api.autohome.com.cn/mobile/decrypt?_appid=qczf&consumerId=qcjr.dev.payment&ciphers=

In [12]:
# 身份证号测试
s='gHIUe3iE35cksqkMaXUz2tC9xp5ihRRuMD9/2qNh05wAdgAAAAE='

# 测试一个手机号试一下
url = '''http://se-j.api.autohome.com.cn/mobile/decrypt?_appid=qczf&consumerId=qcjr.dev.payment&ciphers=%(1)s'''%{'1':s}
req = urllib2.Request(url)
res_data = urllib2.urlopen(req)
json_str = res_data.read()
json_obj = json.loads(json_str)

print json_obj['result'][0]['mobile']

320830198611171216


In [ ]:
id_mingwen=[]
for id_i in data_id['id_urlencode_last']:
    url = '''http://se-j.api.autohome.com.cn/mobile/decrypt?_appid=qczf&consumerId=qcjr.dev.payment&ciphers=%(1)s'''%{'1':id_i}
    req = urllib2.Request(url)
    res_data = urllib2.urlopen(req)
    json_str = res_data.read()
    json_obj = json.loads(json_str)
    id_mingwen.append(json_obj['result'][0]['mobile'])


data_id['id_mingwen']=id_mingwen

In [ ]:
data_id.head()

## 身份信息解析

* 前1、2位数字表示：所在省份的代码；
* 第3、4位数字表示：所在城市的代码；
* 第5、6位数字表示：所在区县的代码；
* 第7~14位数字表示：出生年、月、日；
* 第15、16位数字表示：所在地的派出所的代码；
* 第17位数字表示性别：奇数表示男性，偶数表示女性；


In [ ]:
s='320830198611171216'
s[-2]

In [ ]:
# s[-2]

sex=[]
age=[]
province=[]
city=[]
country=[]

for i in data_id['id_mingwen']:
    sex.append(i[-2])
    age.append(i[6:10])
    province.append(i[0:2])
    city.append(i[0:4])
    country.append(i[0:6])


data_id['sex_id']=sex
data_id['age_id']=age
data_id['province_id']=province
data_id['city_id']=city
data_id['country_id']=country

## 导出身份证明文数据

In [ ]:
# load data
data_id[['mobile_hash','id_initial','sex_id','age_id','province_id','city_id','country_id']].to_csv('data_id_0106.txt'
                ,sep = ','
                ,index = False
                ,encoding = 'utf-8')

# 银行卡号解密

In [46]:
# 身份证号测试
s='P308694845'

# 测试一个手机号试一下
url = '''http://pay.autohome.com.cn/washing-number/realName/genHash.htm?_appid=qczf&consumerId=qcjr.dev.payment&cardNo=%(1)s'''%{'1':s}
req = urllib2.Request(url)
res_data = urllib2.urlopen(req)
json_str = res_data.read()
json_obj = json.loads(json_str)

print json_obj['cardNo']

EpGijZZ0+tP0moiN60YQZ0adVduULtPL4iTdOkNiEP0AdgAAAAE=


In [43]:
# 筛选非空身份证号, 并整体去重
data_card=data_set[data_set.card_initial.notnull()][['mobile_hash','card_initial']].drop_duplicates()
data_card.head()

,mobile_hash,card_initial
0,0000f50fb74f427ffa60daf04e43effe67068da9d30c06ea911bdea7bbfa1bb6,P308694845
2,00022aecf18524455efc768692309abc5cb5f613bf4e627d49ad27e293eef80e,P302539216
7,000b6d48b1f3bac3a4c817927da614d6aae73824e5ef9bbb296f9a4fb3124ff8,P302412584
10,000d598aa2aa27deaa913652163ef18d00bbe74588abdcfe06d00d6e3d315d94,P293474905
15,0012f47b02b3a24183fd552c49ca9ec1059a3aa3573d677e552ef503a89c7771,P308851532


## 原始转可逆密文

In [ ]:
card_miwen=[]
for card_i in data_card['card_initial']:
    url = '''http://pay.autohome.com.cn/washing-number/realName/genHash.htm?_appid=qczf&consumerId=qcjr.dev.payment&cardNo=%(1)s'''%{'1':card_i}
    req = urllib2.Request(url)
    res_data = urllib2.urlopen(req)
    json_str = res_data.read()
    json_obj = json.loads(json_str)
    card_miwen.append(json_obj['cardNo'])


data_card['card_miwen']=card_miwen
data_card.head()

## 可逆密文urlencode

In [108]:
# 测试
a={}
a['name']='SGRiv2ClYhke8vPH4YJm8IdGD/ngWF034jYPGxcu/FwAdgAAAAE='

s={'name':'SGRiv2ClYhke8vPH4YJm8IdGD/ngWF034jYPGxcu/FwAdgAAAAE='}
s
urllib.urlencode(a)
# urlencode(s)

'name=SGRiv2ClYhke8vPH4YJm8IdGD%2FngWF034jYPGxcu%2FFwAdgAAAAE%3D'

In [ ]:
data_card=data_card.reset_index()
card_urlencode=[]
for i in range(data_card.shape[0]):
    i_dict={}
    i_dict['name']=data_card['card_miwen'][i]
    card_urlencode.append(urllib.urlencode(i_dict))
    

data_card['card_urlencode']=card_urlencode

In [ ]:
# 去掉字典的key值
a=[]
for i in data_card['card_urlencode']:
    a.append(i.split('=')[1])


data_card['card_urlencode_last']=a

## 可逆密文转明文

In [83]:
# 银行卡测试
s='0%2FhZ9ybAmBK8ejYvq0MEON7VFlcqrd5nnKk0vkW6IvcAdgAAAAE%3D'

# 测试一个手机号试一下
url = '''http://se-j.api.autohome.com.cn/mobile/decrypt?_appid=qczf&consumerId=qcjr.dev.payment&ciphers=%(1)s'''%{'1':s}
req = urllib2.Request(url)
res_data = urllib2.urlopen(req)
json_str = res_data.read()
json_obj = json.loads(json_str)

print json_obj['result'][0]['mobile']

140502198909180515


In [51]:
card_mingwen=[]
for card_i in data_card['card_urlencode_last']:
    url = '''http://se-j.api.autohome.com.cn/mobile/decrypt?_appid=qczf&consumerId=qcjr.dev.payment&ciphers=%(1)s'''%{'1':card_i}
    req = urllib2.Request(url)
    res_data = urllib2.urlopen(req)
    json_str = res_data.read()
    json_obj = json.loads(json_str)
    card_mingwen.append(json_obj['result'][0]['mobile'])


data_card['card_mingwen']=card_mingwen

19

In [ ]:
data_card.head()

## 卡信息解析

* 1-6是卡BIN号
* 7-10是地区码
* 11-18是顺序号，系统自动生成,无重复
* 19位是校验位。

In [57]:
s='6217000010067732985'
s[6:10]

'0010'

In [ ]:
# s[-2]

card_bin=[]
card_city=[]

for i in data_card['card_mingwen']:
    card_bin.append(i[0:6])
    card_city.append(i[6:10])


data_card['card_bin_id']=card_bin
data_card['card_city_id']=card_city


## 导出银行卡明文数据

In [ ]:
# load data
data_card[['mobile_hash','card_initial','card_mingwen']].to_csv('data_card_0106.txt'
                ,sep = ','
                ,index = False
                ,encoding = 'utf-8')

In [59]:
# load data
card_index=pd.read_table('card_bin_index.txt',sep='\t',header=0,names=['bin_id','card_type','card_name'])
card_index.head()

,bin_id,card_type,card_name
0,694301,借记卡,长沙市商业银行
1,690755,储蓄卡,招商
2,690755,储蓄卡,招商
3,685800,借记卡,广东发展银行
4,685800,信用卡,广东发展银行
